In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp drive/"My Drive"/"Colab Notebooks"/dataset.zip /tmp/dataset.zip

In [ ]:
import os
import zipfile
import numpy as np
import pandas as pd
import cv2

In [ ]:
local_zip = '/tmp/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')

In [ ]:
from keras.models import load_model
classifier = load_model('model.h5')
classifier.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory='/tmp/dataset/test_set',
    target_size=(64, 64),
    color_mode='rgb',
    batch_size=32,
    class_mode=None,
    shuffle=False,
    seed=47
)

In [ ]:
predicted_class_indices=pred.argmax(axis=-1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
#results.to_csv("results.csv",index=False)

In [ ]:
def ndvi_prediction(filepath):
    img = cv2.imread(filepath, 1)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    for x in range(0,256):
        for y in range(0,256):
            if img_hsv[x][y][1] < 51:
                img[x][y][0]=0
                img[x][y][1]=0
                img[x][y][2]=0
    
    unhealthy = 0
    actual_pixels = 0
    for x in range(0,256):
        for y in range(0,256):
            if img[x][y][0] != 0 and img[x][y][1] != 0 and img[x][y][2] != 0:
                actual_pixels += 1
                nir = ((1.8474519*img[x][y][2])-(0.1936929*img[x][y][1])+(0.12401134*img[x][y][0]))
                if (nir - img[x][y][2]) / (nir + img[x][y][2]) < 0.2463804:
                    unhealthy += 1

    if unhealthy / actual_pixels >= 0.19:
        return "unhealthy"
    else:
        return "healthy"

In [ ]:
df = pd.read_csv("sample_data/results.csv")

In [ ]:
df['NDVI_Prediction'] = [ndvi_prediction(os.path.join("/tmp/dataset/test_set", filename)) for filename in df['Filename']]

In [ ]:
result = []
ndvi = df['NDVI_Prediction']
cnn = df['Predictions']
for i in range(len(ndvi)):
    if ndvi[i] == "unhealthy" and cnn[i] == "healthy":
        result.append("unhealthy")
    else:
        result.append(cnn[i])

In [ ]:
df['Final_Output'] = result

In [ ]:
df.columns = ['Filenames', 'CNN_Predictions', 'NDVI_Predictions', 'Final_Output']

In [ ]:
df.to_csv("results.csv")